<hr style="border:2px solid RosyBrown"> </hr>
<hr style="border:1px solid Wheat"> </hr>

# Constrained Markov Clustering

<hr style="border:1px solid Wheat"> </hr>
<hr style="border:2px solid RosyBrown"> </hr>

Load all packages and modules

In [49]:
%load_ext autoreload
%autoreload 2

#------------------------------------------------------------------------------#
# Import modules
from sklearn.metrics.cluster import normalized_mutual_info_score
import sys
import os
import numpy as np
import pandas as pd
#------------------------------------------------------------------------------#

#------------------------------------------------------------------------------#
# Import custom classes and functions
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)
    
from models.data_gen import dataGen
from models.CoMaC import CoMaC

from utils.callback import save_results
from utils.helperFunc import partition_to_labels, generate_int_labels
from utils.plotting import show_clustering, show_transition_prob
from utils.dataloader import load_dataset

#------------------------------------------------------------------------------#

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
#------------------------------------------------------------------------------#
# Select from different datasets
#------------------------------------------------------------------------------#

X, labels_true = load_dataset('IRIS')

labels_true = generate_int_labels(labels_true)
M = len(np.unique(labels_true))

print(f'{dataset_str} dataset with {X.shape[1]} features, {X.shape[0]} samples and {M} classes.')


IRIS dataset with 4 features, 150 samples and 3 classes.


In [85]:
#------------------------------------------------------------------------------#
# Parameters

knns=20
restarts=5
percentage=0.2
final_beta=0.0
#------------------------------------------------------------------------------#

In [86]:
#------------------------------------------------------------------------------#
# Initialize and generate constraints

# create an instance of the comac class
comac = CoMaC(M=M, knns=knns, restarts=restarts)

### exhaustive constraints 
# retrieved from partition level side information
# |X_sampled|/|X| = percentage
# generate M (must link) and N (cannot link) from X_sampled

# comac.constraints(X, labels_true, percentage=percentage,
#                   wrong_percentage=0, ClassLabels=2)

### non-exhaustive constraints
# (|M|+|N|)/|X| = percentage

comac.random_constraints(X, labels_true, percentage, flag_connected=True)
print('Total number of constraints: ', len(comac.M_must)+len(comac.M_cannot))
#------------------------------------------------------------------------------#

Number of additional constraints due to constraint propagation:  4
Total number of constraints:  30


In [71]:
#------------------------------------------------------------------------------#
# Sequential algorithm
final_beta = 0.5 
print('~'*80 + f'\n Sequential Algorithm: beta = {final_beta}')
cost_seq, V_seq = comac.cluster_seq(X, beta=final_beta)

#------------------------------------------------------------------------------#

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 Sequential Algorithm: beta = 0.5
*************************Clustering Finished!*************************


In [72]:
#------------------------------------------------------------------------------#
labels_seq = partition_to_labels(V_seq) 
NMI_seq = normalized_mutual_info_score(labels_true, labels_seq)

print(f'Sequential NMI = {NMI_seq} (beta={final_beta})')
#------------------------------------------------------------------------------#

Sequential NMI = 0.9701911554558436 (beta=0.5)


In [73]:
#------------------------------------------------------------------------------#
# Annealing algorithm

print('~'*80 + f'\n Annealing Algorithm: final beta = {final_beta}')
cost_ann, V_ann, beta_vec = comac.cluster_ann(X, final_beta=final_beta, step_size=-0.5)

#------------------------------------------------------------------------------#

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 Annealing Algorithm: final beta = 0.5
Starting with beta = 1
*************************Clustering Finished!*************************
Starting with beta = 0.5
*************************Clustering Finished!*************************


In [74]:
#------------------------------------------------------------------------------#

labels_ann = []
NMI_ann = np.zeros_like(beta_vec)

for idx, beta in enumerate(beta_vec):
    labels_ann.append( partition_to_labels(V_ann[idx, :, :]) )
    NMI_ann[idx] = normalized_mutual_info_score(labels_true,
                                                labels_ann[-1])
    print(f'NMI = {NMI_ann[idx]} (beta={beta}) \n')
    
#------------------------------------------------------------------------------#

NMI = 1.0 (beta=1.0) 

NMI = 0.9701911554558436 (beta=0.5) 



<hr style="border:2px solid DarkKhaki"> </hr>
